In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from keras.datasets import mnist

In [25]:
((x_train, y_train), (x_test, y_test))= mnist.load_data()

In [26]:
only_zeros = x_train[y_train == 0]

In [27]:
x_train.shape, only_zeros.shape

((60000, 28, 28), (5923, 28, 28))

In [28]:
x_train = x_train / 255

In [29]:
x_train = x_train.reshape(-1, 28, 28, 1) * 2. - 1.

In [30]:
x_train.min(), x_train.max()

(np.float64(-1.0), np.float64(1.0))

In [31]:
only_zeros.shape

(5923, 28, 28)

In [39]:
generator = tf.keras.models.Sequential([
    tf.keras.layers.Dense(7 * 7 * 128, input_shape=[100]),
    tf.keras.layers.Reshape([ 7, 7, 128]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2DTranspose(64, kernel_size=(5,5),
                                    strides=(2,2), activation="relu", padding="same"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2DTranspose(1, kernel_size=(5,5),
                                    strides=(2,2), activation="tanh", padding="same"),

])

In [33]:
discriminator = tf.keras.models.Sequential([

             tf.keras.layers.Conv2D(64, kernel_size=(5,5),
                                    strides=(2,2), activation="relu", input_shape=[28, 28, 1]),

             tf.keras.layers.Dropout(0.4),
            tf.keras.layers.Conv2DTranspose(64, kernel_size=(5,5),
                                    strides=(2,2), activation="relu"),
             tf.keras.layers.Dropout(0.4),
             tf.keras.layers.Flatten(),
             tf.keras.layers.Dense(1, activation="sigmoid")

])

In [40]:
GAN = tf.keras.models.Sequential([generator, discriminator])

In [41]:
discriminator.trainable = False

In [42]:
GAN.compile(loss='binary_crossentropy', optimizer='adam')

In [43]:
discriminator.compile(loss='binary_crossentropy', optimizer='adam')

In [44]:
batch_size = 32

In [45]:
dataset = tf.data.Dataset.from_tensor_slices(only_zeros).shuffle(buffer_size=1000)

In [46]:
new_dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(1)


In [47]:
new_dataset

<_PrefetchDataset element_spec=TensorSpec(shape=(32, 28, 28), dtype=tf.uint8, name=None)>

In [ ]:
EPOCHS = 20

for epoch in range(EPOCHS):
    print(f"Suntem in epoca {epoch + 1}")

    for x_batch in new_dataset:
        noise = tf.random.normal(shape=(batch_size, 100))
        gen_images = generator(noise) # fake images

        # Expand dimensions of x_batch to match gen_images
        x_batch_expanded = tf.expand_dims(tf.dtypes.cast(x_batch, tf.float32), axis=-1)

        x_fake_and_real = tf.concat([gen_images, x_batch_expanded], axis=0)
        y1 = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)


        discriminator.trainable = True
        discriminator.train_on_batch(x_fake_and_real, y1)

        discriminator.trainable = False

        noise = tf.random.normal(shape=(batch_size, 100))
        y2 = tf.constant([[1.]] * batch_size)



        GAN.train_on_batch(noise, y2)

Suntem in epoca 1
Suntem in epoca 2
Suntem in epoca 3
Suntem in epoca 4
Suntem in epoca 5
Suntem in epoca 6
Suntem in epoca 7
Suntem in epoca 8


In [ ]:
noise = tf.random.normal(shape=(10, 100))
plt.imshow(noise)

In [ ]:
images =  generator(noise)
for im in images:
    plt.imshow(im)
    plt.show()